In [23]:
import numpy as np
import plotly.graph_objects as go
import csv
import scipy as sp

In [9]:
def read_data(file_path):

    with open(file_path) as f:
        reader = csv.reader(f)
        data = []
        for column in reader:
            column[0] = column[0].replace('(', '').replace("'" , '')
            column[2] = column[2].replace(')', '').replace("'" , '')
            column[1] = column[1].replace("'" , '')
            column[0] = float(column[0])
            column[1] = float(column[1])
            column[2] = float(column[2])
            data.append(column)
        data = np.array(data)
    return data
        # print(data)

In [19]:
initial = read_data('../data/0_31G.csv')
second = read_data('../data/0_32G.csv')
third = read_data('../data/0_9G.csv')



In [14]:
def plot_pos(data, title, proj):
    if proj=='xy':
        fig = go.Figure(data=go.Scatter(x=data[:,0], y=data[:,1], mode='markers'))
        fig.update_layout(title=title, xaxis_title='x', yaxis_title='y')
        fig.update_layout(
            width=500,
            height=500,
            margin=dict(
                l=50,
                r=50,
                b=100,
                t=100,
                pad=4
            ),
            xaxis=dict(
                showgrid=True,
                zeroline=True,
                showline=True,
                showticklabels=True,
                scaleratio=1
            ),
            yaxis=dict(
                showgrid=True,
                zeroline=True,
                showline=True,
                showticklabels=True,
                scaleratio=1
            )
        )
        fig.show()
    elif proj=='xz':
        fig = go.Figure(data=go.Scatter(x=data[:,0], y=data[:,2], mode='markers'))
        fig.update_layout(title=title, xaxis_title='x', yaxis_title='z')
        fig.update_layout(
            width=500,
            height=500,
            margin=dict(
                l=50,
                r=50,
                b=100,
                t=100,
                pad=4
            ),
            xaxis=dict(
                showgrid=True,
                zeroline=True,
                showline=True,
                showticklabels=True,
                scaleratio=1
            ),
            yaxis=dict(
                showgrid=True,
                zeroline=True,
                showline=True,
                showticklabels=True,
                scaleratio=1
            )
        )
        fig.show()
    elif proj=='yz':
        fig = go.Figure(data=go.Scatter(x=data[:,1], y=data[:,2], mode='markers'))
        fig.update_layout(title=title, xaxis_title='y', yaxis_title='z')
        fig.update_layout(
            width=500,
            height=500,
            margin=dict(
                l=50,
                r=50,
                b=100,
                t=100,
                pad=4
            ),
            xaxis=dict(
                showgrid=True,
                zeroline=True,
                showline=True,
                showticklabels=True,
                scaleratio=1
            ),
            yaxis=dict(
                showgrid=True,
                zeroline=True,
                showline=True,
                showticklabels=True,
                scaleratio=1
            )
        )
        fig.show()
    elif proj=='3d':
        fig = go.Figure(data=[go.Scatter3d(x=data[:,0], y=data[:,1], z=data[:,2], mode='markers')])
        fig.update_layout(title=title, scene_aspectmode='cube', scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'), width=700, height=700)
        fig.show()
    else:
        print('Invalid projection')

In [21]:
plot_pos(initial, '0_31G', '3d')
plot_pos(second, '0_32G', 'xy')
plot_pos(third, '0_9G', '3d')


In [16]:
#create lines from origin to each point
origin = np.array([0,0,-500])
lines = np.zeros((len(data), 2, 3))
for i in range(len(data)):
    lines[i][0] = origin
    lines[i][1] = data[i]
print(lines)

NameError: name 'data' is not defined

In [42]:
angle = cherenkov_angle(data)

0
[30. 25.  0.]
1
[-15.   0.   0.]
2
[5. 5. 0.]
3
[ 15. -15.   0.]
4
[-30.   5.   0.]
5
[15.  5.  0.]
6
[ 15. -30.   0.]
7
[ 5. 15.  0.]
8
[-40.  -5.   0.]
9
[20. 20.  0.]
10
[-10.   0.   0.]
11
[ 10. -35.   0.]
12
[5. 0. 0.]
13
[10. 15.  0.]
14
[-25. -15.   0.]
15
[0. 0. 0.]
16
[-5.  5.  0.]
17
[ 5. 30.  0.]
18
[5. 0. 0.]
19
[ 10. -30.   0.]
20
[-20.  25.   0.]
21
[ 15. -10.   0.]
22
[0. 0. 0.]


C:\Users\Stephanie\AppData\Local\Temp\ipykernel_3324\335807945.py:14: RuntimeWarning:

invalid value encountered in scalar divide



# Plot the cherenkov angle

In [ ]:
def momentum(E, m):
    return np.sqrt(E**2 - m**2)

In [25]:
def fit_circle(data):
    x, y = data[:,0], data[:,1]
    x_m, y_m = np.mean(x), np.mean(y)

    def calc_R(xc, yc):
        """ calculate the distance of each 2D points from the center (xc, yc) """
        return np.sqrt((x-xc)**2 + (y-yc)**2)

    def f(c):
        """ calculate the algebraic distance between the data points and the mean circle centered at c=(xc, yc) """
        Ri = calc_R(*c)
        return Ri - Ri.mean()

    center_estimate = x_m, y_m
    center, _ = sp.optimize.leastsq(f, center_estimate)

    xc, yc = center
    Ri       = calc_R(*center)
    R       = Ri.mean()
    return xc, yc, R

In [30]:
yc, zc, Rc = fit_circle(third)
print(f"Center at ({yc}, {zc}) with radius {Rc}")
print(yc, zc, Rc)

Center at (-0.23599393838940805, 0.2430185736353024) with radius 109.05464897964558
-0.23599393838940805 0.2430185736353024 109.05464897964558


In [31]:
def plot_circle(xc, yc, R):
    theta_fit = np.linspace(0, 2*np.pi, 180)
    x_fit = xc + R*np.cos(theta_fit)
    y_fit = yc + R*np.sin(theta_fit)
    fig = go.Figure(data=go.Scatter(x=x_fit, y=y_fit, mode='lines'))
    fig.update_layout(title='0_9G', xaxis_title='y', yaxis_title='z')
    fig.update_layout(
        width=500,
        height=500,
        margin=dict(
            l=50,
            r=50,
            b=100,
            t=100,
            pad=4
        ),
        xaxis=dict(
            showgrid=True,
            zeroline=True,
            showline=True,
            showticklabels=True,
            scaleratio=1
        ),
        yaxis=dict(
            showgrid=True,
            zeroline=True,
            showline=True,
            showticklabels=True,
            scaleratio=1
        )
    )
    fig.show()



In [32]:
plot_circle(yc, zc, Rc)